# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import json
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
import pickle
path = "/content/drive/MyDrive/COMSATS University Islamabad/Final Year Project/Dataset Files/ML Dataset/DyxsisML Auto-generated Complete Dataset.json"

# Generate Data for 4th Model

In [ ]:
headers = ['timestamp', 'gender', 'questions', 'timeToFinish', 'level', 'dyslexic']
sub_headers = ['difficulty', 'clickCount', 'hits', 'miss', 'score',
               'missrate', 'accuracy', 'time', ]
genders = ['male', 'female']
levels = ['preschooler', 'learner', 'elementary']
difficulty = ['easy', 'medium', 'hard']
dyslexic = ['yes', 'no']


def getgender():
    return genders[random.randint(0, 1)]


def getlevel():
    return levels[random.randint(0, 2)]


def getdifficulty():
    return difficulty[random.randint(0, 2)]


def isdyslexic():
    return dyslexic[random.randint(0, 1)]



Easy_4 = []
Medium_4 = []
Hard_4 = []
labels_4 = []

for i in range(15000):
    ez = []
    mid = []
    hrd = []
    row = dict.fromkeys(headers)
    row['gender'] = getgender()
    row['timeToFinish'] = random.randint(420, 1800)
    row['questions'] = []
    row['dyslexic'] = isdyslexic()
    for j in range(15):
        ques = dict.fromkeys(sub_headers)
        ques['difficulty'] = getdifficulty()
        ques['clickCount'] = random.randint(1, 10)
        ques['hits'] = random.randint(0, 1)
        ques['miss'] = 1 - ques['hits']
        ques['score'] = ques['hits']
        ques['missrate'] = round(ques['miss'] / ques['clickCount'], 2)
        ques['accuracy'] = ques['hits'] / ques['clickCount']
        ques['time'] = random.randint(10, 30)
        row['questions'].append(ques)

        if ques['difficulty'].strip() == 'easy':
            ez = ez + [ques['clickCount'], ques['hits'], ques['miss'],
                       ques['score'],
                       ques['missrate'], ques['accuracy'], ques['time']]
        elif ques['difficulty'].strip() == 'medium':
            mid = mid + [ques['clickCount'], ques['hits'], ques['miss'],
                         ques['score'],
                         ques['missrate'], ques['accuracy'], ques['time']]
        else:
            hrd = hrd + [ques['clickCount'], ques['hits'], ques['miss'],
                         ques['score'],
                         ques['missrate'], ques['accuracy'], ques['time']]

    Easy_4.append([row['gender'], row['timeToFinish']]+ ez )
    Medium_4.append([row['gender'], row['timeToFinish']] +mid)
    Hard_4.append([row['gender'], row['timeToFinish']] + hrd)
    labels_4.append(row['dyslexic'])

# Load Train Data from JSON File

In [ ]:
with open(path,"r",encoding="utf-8") as f:
    data = json.load(f)

# Max Padding

In [ ]:
max_padding = 60

# Splitting Train Dataset into Arrays

In [ ]:
Easy_train = []
Medium_train = []
Hard_train = []
labels_train = []

for row in data:
    ez = []
    mid = []
    hrd = []
    for ques in row['questions']:
        if ques['difficulty'].strip() == 'easy':
            ez = ez + [ques['clickCount'], ques['hits'], ques['miss'],
                       ques['score'],
                       ques['missrate'], ques['accuracy'], ques['time']]
        elif ques['difficulty'].strip() == 'medium':
            mid = mid + [ques['clickCount'], ques['hits'], ques['miss'],
                         ques['score'],
                         ques['missrate'], ques['accuracy'], ques['time']]
        else:
            hrd = hrd + [ques['clickCount'], ques['hits'], ques['miss'],
                         ques['score'],
                         ques['missrate'], ques['accuracy'], ques['time']]

    Easy_train.append([row['gender'], row['timeToFinish']] + ez)
    Medium_train.append([row['gender'], row['timeToFinish']] + mid)
    Hard_train.append([row['gender'], row['timeToFinish']] + hrd)
    labels_train.append(row['dyslexic'])

In [ ]:
for item in Easy_train:
    print(item)

Streaming output truncated to the last 5000 lines.
['male', 1460, 9, 1, 0, 1, 0.0, 0.1111111111111111, 28, 9, 1, 0, 1, 0.0, 0.1111111111111111, 11, 5, 1, 0, 1, 0.0, 0.2, 19, 4, 1, 0, 1, 0.0, 0.25, 19]
['male', 1386, 2, 0, 1, 0, 0.5, 0.0, 28, 6, 1, 0, 1, 0.0, 0.16666666666666666, 22, 5, 1, 0, 1, 0.0, 0.2, 26, 3, 1, 0, 1, 0.0, 0.3333333333333333, 27]
['male', 1653, 8, 0, 1, 0, 0.12, 0.0, 25, 4, 1, 0, 1, 0.0, 0.25, 13, 2, 1, 0, 1, 0.0, 0.5, 14, 6, 1, 0, 1, 0.0, 0.16666666666666666, 15]
['male', 530, 8, 0, 1, 0, 0.12, 0.0, 23, 1, 1, 0, 1, 0.0, 1.0, 14, 9, 1, 0, 1, 0.0, 0.1111111111111111, 14, 4, 0, 1, 0, 0.25, 0.0, 16, 10, 1, 0, 1, 0.0, 0.1, 23, 9, 1, 0, 1, 0.0, 0.1111111111111111, 27]
['female', 1300, 3, 0, 1, 0, 0.33, 0.0, 28, 8, 1, 0, 1, 0.0, 0.125, 16, 8, 1, 0, 1, 0.0, 0.125, 22, 7, 1, 0, 1, 0.0, 0.14285714285714285, 25, 9, 1, 0, 1, 0.0, 0.1111111111111111, 22]
['male', 1352, 4, 1, 0, 1, 0.0, 0.25, 29, 4, 0, 1, 0, 0.25, 0.0, 29, 10, 1, 0, 1, 0.0, 0.1, 17, 6, 0, 1, 0, 0.17, 0.0, 17]
['f

In [ ]:
Easy_train[0]

['female',
 1049,
 1,
 1,
 0,
 1,
 0.0,
 1.0,
 11,
 8,
 1,
 0,
 1,
 0.0,
 0.125,
 11,
 3,
 0,
 1,
 0,
 0.33,
 0.0,
 21,
 5,
 0,
 1,
 0,
 0.2,
 0.0,
 10,
 8,
 0,
 1,
 0,
 0.12,
 0.0,
 13]

# Encoding  Train Labels

In [ ]:
le = LabelEncoder()
en_labels_train = le.fit_transform(labels_train)
le.classes_

array(['no', 'yes'], dtype='<U3')

In [ ]:
en_labels_train


array([1, 0, 0, ..., 1, 0, 1])

In [ ]:
labels_train[:10]

['yes', 'no', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'yes', 'no']

Hence we have yes = 1, and no= 0

# Encoding Labels for 4th Model

In [ ]:
le = LabelEncoder()
en_labels_4 = le.fit_transform(labels_4)
le.classes_

array(['no', 'yes'], dtype='<U3')

# Custom Functions

In [ ]:
def encode_gender(data_array):
    for row in data_array:
        if row[0].lower().strip() == "male":
            row[0] = 1
        else:
            row[0] = 0
    return data_array

# Data Wrangling for 4th Model

## Easy_4 Dataset

In [ ]:
encoded_easy_4  = encode_gender(Easy_4) 

In [ ]:
padded_easy_4 = pad_sequences(encoded_easy_4,padding="post",maxlen=max_padding,truncating="post")

In [ ]:
df_easy_4 = pd.DataFrame(data=padded_easy_4)
df_easy_4.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,1,1619,5,0,1,0,0,0,26,4,0,1,0,0,0,12,10,1,0,1,0,0,19,6,0,1,0,0,0,17,5,1,0,1,0,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,910,10,1,0,1,0,0,10,6,1,0,1,0,0,24,5,1,0,1,0,0,28,3,1,0,1,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1585,8,1,0,1,0,0,27,6,0,1,0,0,0,11,8,1,0,1,0,0,22,6,0,1,0,0,0,30,7,0,1,0,0,0,27,10,0,1,0,0,0,19,4,1,0,1,0,0,26,6,1,0,1,0,0,10,9,1
3,1,1016,1,1,0,1,0,1,21,8,0,1,0,0,0,30,10,1,0,1,0,0,24,7,0,1,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1685,4,0,1,0,0,0,16,10,0,1,0,0,0,16,2,1,0,1,0,0,19,1,0,1,0,1,0,15,6,1,0,1,0,0,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# df_easy_4.insert(max_padding,max_padding,en_labels_4,True)
# df_easy_4.head()

In [ ]:
# X_easy_4 = df_easy_4.iloc[:,:-1]
# X_easy_4.head()

In [ ]:
# Y_easy_4 = df_easy_4.iloc[:,-1]
# Y_easy_4.head()

In [ ]:
# X_train_easy_4, X_test_easy_4, Y_train_easy_4, Y_test_easy_4 = train_test_split(X_easy_4, Y_easy_4, stratify=Y_easy_4,random_state=1,test_size=0.2)

In [ ]:
# print("Train set X size: ",X_train_easy_4.shape)
# print("Train set Y size: ",Y_train_easy_4.shape)
# print("Test set X size: ",X_test_easy_4.shape)
# print("Test set Y size: ",Y_test_easy_4.shape)

## Medium Test Dataset

In [ ]:
encoded_medium_4  = encode_gender(Medium_4) 

In [ ]:
padded_medium_4 = pad_sequences(encoded_medium_4,padding="post",maxlen=max_padding,truncating="post")

In [ ]:
df_medium_4 = pd.DataFrame(data=padded_medium_4)
df_medium_4.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,1,1619,9,0,1,0,0,0,24,4,0,1,0,0,0,10,1,1,0,1,0,1,29,4,0,1,0,0,0,29,4,0,1,0,0,0,30,7,0,1,0,0,0,20,7,0,1,0,0,0,13,0,0,0,0,0,0,0,0,0
1,1,910,2,1,0,1,0,0,23,3,0,1,0,0,0,30,5,1,0,1,0,0,30,2,1,0,1,0,0,21,1,1,0,1,0,1,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1585,5,0,1,0,0,0,20,3,0,1,0,0,0,11,7,1,0,1,0,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1016,3,0,1,0,0,0,10,10,1,0,1,0,0,27,9,0,1,0,0,0,23,7,0,1,0,0,0,24,2,1,0,1,0,0,11,9,1,0,1,0,0,19,3,1,0,1,0,0,24,0,0,0,0,0,0,0,0,0
4,0,1685,6,0,1,0,0,0,15,4,1,0,1,0,0,14,8,0,1,0,0,0,19,4,1,0,1,0,0,12,3,1,0,1,0,0,27,6,1,0,1,0,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# df_medium_4.insert(max_padding,max_padding,en_labels_4,True)
# df_medium_4.head()

In [ ]:
# X_medium_4 = df_medium_4.iloc[:,:-1]
# X_medium_4.head()

In [ ]:
# Y_medium_4 = df_medium_4.iloc[:,-1]
# Y_medium_4.head()

In [ ]:
# X_train_medium_4, X_test_medium_4, Y_train_medium_4, Y_test_medium_4 = train_test_split(X_medium_4, Y_medium_4, stratify=Y_medium_4,random_state=1,test_size=0.2)

In [ ]:
# print("Train set X size: ",X_train_medium_4.shape)
# print("Train set Y size: ",Y_train_medium_4.shape)
# print("Test set X size: ",X_test_medium_4.shape)
# print("Test set Y size: ",Y_test_medium_4.shape)

## Hard Test Dataset

In [ ]:
encoded_hard_4  = encode_gender(Hard_4) 

In [ ]:
padded_hard_4 = pad_sequences(encoded_hard_4,padding="post",maxlen=max_padding,truncating="post")

In [ ]:
df_hard_4 = pd.DataFrame(data=padded_hard_4)
df_hard_4.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,1,1619,9,1,0,1,0,0,29,5,0,1,0,0,0,11,5,1,0,1,0,0,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,910,4,1,0,1,0,0,16,9,0,1,0,0,0,24,1,0,1,0,1,0,22,2,0,1,0,0,0,25,7,0,1,0,0,0,27,9,1,0,1,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1585,6,0,1,0,0,0,25,10,0,1,0,0,0,16,3,0,1,0,0,0,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1016,1,1,0,1,0,1,21,10,0,1,0,0,0,15,6,0,1,0,0,0,18,8,0,1,0,0,0,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1685,5,0,1,0,0,0,27,1,1,0,1,0,1,17,7,1,0,1,0,0,29,5,1,0,1,0,0,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# df_hard_4.insert(max_padding,max_padding,en_labels_4,True)
# df_hard_4.head()

In [ ]:
# X_hard_4 = df_hard_4.iloc[:,:-1]
# X_hard_4.head()

In [ ]:
# Y_hard_4 = df_hard_4.iloc[:,-1]
# Y_hard_4.head()

In [ ]:
# X_train_hard_4, X_test_hard_4, Y_train_hard_4, Y_test_hard_4 = train_test_split(X_hard_4, Y_hard_4, stratify=Y_hard_4,random_state=1,test_size=0.2)

In [ ]:
# print("Train set X size: ",X_train_hard_4.shape)
# print("Train set Y size: ",Y_train_hard_4.shape)
# print("Test set X size: ",X_test_hard_4.shape)
# print("Test set Y size: ",Y_test_hard_4.shape)

# Easy Dataset

## Encode Gender by Using Manual Hot Vector Encoding

In [ ]:
encoded_easy_input  = encode_gender(Easy_train) 

## Padding Arrays With Zeros

In [ ]:
padded_easy_input = pad_sequences(encoded_easy_input[0],padding="post",maxlen=max_padding,truncating="post")

ValueError: ignored

## Creating X,Y DataFrames

In [ ]:
df_easy = pd.DataFrame(data=padded_easy_input)
df_easy.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,0,1049,1,1,0,1,0,1,11,8,1,0,1,0,0,11,3,0,1,0,0,0,21,5,0,1,0,0,0,10,8,0,1,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1773,4,1,0,1,0,0,29,5,0,1,0,0,0,12,7,1,0,1,0,0,14,7,0,1,0,0,0,12,5,1,0,1,0,0,25,2,0,1,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1375,6,1,0,1,0,0,16,5,0,1,0,0,0,18,8,1,0,1,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1077,3,1,0,1,0,0,15,7,1,0,1,0,0,12,10,1,0,1,0,0,26,4,0,1,0,0,0,17,7,0,1,0,0,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,797,9,1,0,1,0,0,15,7,1,0,1,0,0,24,7,1,0,1,0,0,20,5,1,0,1,0,0,28,9,1,0,1,0,0,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_easy.insert(max_padding,max_padding,en_labels_train,True)
df_easy.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0,1049,1,1,0,1,0,1,11,8,1,0,1,0,0,11,3,0,1,0,0,0,21,5,0,1,0,0,0,10,8,0,1,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,1773,4,1,0,1,0,0,29,5,0,1,0,0,0,12,7,1,0,1,0,0,14,7,0,1,0,0,0,12,5,1,0,1,0,0,25,2,0,1,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1375,6,1,0,1,0,0,16,5,0,1,0,0,0,18,8,1,0,1,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1077,3,1,0,1,0,0,15,7,1,0,1,0,0,12,10,1,0,1,0,0,26,4,0,1,0,0,0,17,7,0,1,0,0,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,797,9,1,0,1,0,0,15,7,1,0,1,0,0,24,7,1,0,1,0,0,20,5,1,0,1,0,0,28,9,1,0,1,0,0,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_easy = df_easy.iloc[:,:-1]
X_easy.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,0,1049,1,1,0,1,0,1,11,8,1,0,1,0,0,11,3,0,1,0,0,0,21,5,0,1,0,0,0,10,8,0,1,0,0,0,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1773,4,1,0,1,0,0,29,5,0,1,0,0,0,12,7,1,0,1,0,0,14,7,0,1,0,0,0,12,5,1,0,1,0,0,25,2,0,1,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1375,6,1,0,1,0,0,16,5,0,1,0,0,0,18,8,1,0,1,0,0,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1077,3,1,0,1,0,0,15,7,1,0,1,0,0,12,10,1,0,1,0,0,26,4,0,1,0,0,0,17,7,0,1,0,0,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,797,9,1,0,1,0,0,15,7,1,0,1,0,0,24,7,1,0,1,0,0,20,5,1,0,1,0,0,28,9,1,0,1,0,0,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
Y_easy = df_easy.iloc[:,-1]
Y_easy.head()

0    1
1    0
2    0
3    1
4    0
Name: 60, dtype: int64

## X,Y Train Test Split 

In [ ]:
X_train_easy, X_test_easy, Y_train_easy, Y_test_easy = train_test_split(X_easy, Y_easy, stratify=Y_easy,random_state=1,test_size=0.2)

In [ ]:
print("Train set X size: ",X_train_easy.shape)
print("Train set Y size: ",Y_train_easy.shape)
print("Test set X size: ",X_test_easy.shape)
print("Test set Y size: ",Y_test_easy.shape)

Train set X size:  (12000, 60)
Train set Y size:  (12000,)
Test set X size:  (3000, 60)
Test set Y size:  (3000,)


# Medium Dataset

## Encode Gender by Using Manual Hot Vector Encoding

In [ ]:
encoded_medium_input = encode_gender(Medium_train)

## Padding Arrays With Zeros

In [ ]:
padded_medium_input = pad_sequences(encoded_medium_input,padding="post",maxlen=max_padding,truncating="post")

## Creating X,Y DataFrames

In [ ]:
df_medium = pd.DataFrame(data=padded_medium_input)
df_medium.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,0,1049,4,0,1,0,0,0,25,9,1,0,1,0,0,16,1,1,0,1,0,1,10,4,0,1,0,0,0,30,5,0,1,0,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1773,10,0,1,0,0,0,24,3,0,1,0,0,0,25,2,0,1,0,0,0,17,5,0,1,0,0,0,27,4,0,1,0,0,0,25,1,0,1,0,1,0,13,6,1,0,1,0,0,10,0,0,0,0,0,0,0,0,0
2,0,1375,9,1,0,1,0,0,24,7,1,0,1,0,0,15,10,1,0,1,0,0,27,3,1,0,1,0,0,25,4,0,1,0,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1077,2,1,0,1,0,0,15,3,1,0,1,0,0,23,1,0,1,0,1,0,17,10,0,1,0,0,0,25,6,0,1,0,0,0,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,797,1,1,0,1,0,1,20,4,1,0,1,0,0,26,10,1,0,1,0,0,17,7,0,1,0,0,0,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_medium.insert(max_padding,max_padding,en_labels_train,True)
df_medium.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0,1049,4,0,1,0,0,0,25,9,1,0,1,0,0,16,1,1,0,1,0,1,10,4,0,1,0,0,0,30,5,0,1,0,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,1773,10,0,1,0,0,0,24,3,0,1,0,0,0,25,2,0,1,0,0,0,17,5,0,1,0,0,0,27,4,0,1,0,0,0,25,1,0,1,0,1,0,13,6,1,0,1,0,0,10,0,0,0,0,0,0,0,0,0,0
2,0,1375,9,1,0,1,0,0,24,7,1,0,1,0,0,15,10,1,0,1,0,0,27,3,1,0,1,0,0,25,4,0,1,0,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1077,2,1,0,1,0,0,15,3,1,0,1,0,0,23,1,0,1,0,1,0,17,10,0,1,0,0,0,25,6,0,1,0,0,0,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,797,1,1,0,1,0,1,20,4,1,0,1,0,0,26,10,1,0,1,0,0,17,7,0,1,0,0,0,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_medium = df_medium.iloc[:,:-1]
X_medium.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,0,1049,4,0,1,0,0,0,25,9,1,0,1,0,0,16,1,1,0,1,0,1,10,4,0,1,0,0,0,30,5,0,1,0,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1773,10,0,1,0,0,0,24,3,0,1,0,0,0,25,2,0,1,0,0,0,17,5,0,1,0,0,0,27,4,0,1,0,0,0,25,1,0,1,0,1,0,13,6,1,0,1,0,0,10,0,0,0,0,0,0,0,0,0
2,0,1375,9,1,0,1,0,0,24,7,1,0,1,0,0,15,10,1,0,1,0,0,27,3,1,0,1,0,0,25,4,0,1,0,0,0,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1077,2,1,0,1,0,0,15,3,1,0,1,0,0,23,1,0,1,0,1,0,17,10,0,1,0,0,0,25,6,0,1,0,0,0,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,797,1,1,0,1,0,1,20,4,1,0,1,0,0,26,10,1,0,1,0,0,17,7,0,1,0,0,0,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
Y_medium = df_medium.iloc[:,-1]
Y_medium.head()

0    1
1    0
2    0
3    1
4    0
Name: 60, dtype: int64

## X,Y Train Test Split 

In [ ]:
X_train_medium, X_test_medium, Y_train_medium, Y_test_medium = train_test_split(X_medium, Y_medium, stratify=Y_medium,random_state=1,test_size=0.2)

In [ ]:
print("Train set X size: ",X_train_medium.shape)
print("Train set Y size: ",Y_train_medium.shape)
print("Test set X size: ",X_test_medium.shape)
print("Test set Y size: ",Y_test_medium.shape)

Train set X size:  (12000, 60)
Train set Y size:  (12000,)
Test set X size:  (3000, 60)
Test set Y size:  (3000,)


# Hard Dataset

## Encode Gender by Using Manual Hot Vector Encoding

In [ ]:
encoded_hard_input = encode_gender(Hard_train)

## Padding Arrays With Zeros

In [ ]:
padded_hard_input = pad_sequences(encoded_hard_input,padding="post",maxlen=max_padding,truncating="post")

## Creating X,Y DataFrames

In [ ]:
df_hard = pd.DataFrame(data=padded_hard_input)
df_hard.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,0,1049,10,0,1,0,0,0,26,5,1,0,1,0,0,22,4,0,1,0,0,0,20,2,0,1,0,0,0,30,6,1,0,1,0,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1773,4,1,0,1,0,0,13,7,1,0,1,0,0,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1375,5,0,1,0,0,0,21,5,0,1,0,0,0,24,7,1,0,1,0,0,16,4,0,1,0,0,0,27,8,1,0,1,0,0,20,2,0,1,0,0,0,18,2,1,0,1,0,0,22,0,0,0,0,0,0,0,0,0
3,0,1077,9,1,0,1,0,0,23,10,0,1,0,0,0,22,3,1,0,1,0,0,21,4,1,0,1,0,0,23,3,0,1,0,0,0,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,797,10,0,1,0,0,0,10,7,0,1,0,0,0,10,9,1,0,1,0,0,20,3,0,1,0,0,0,23,10,0,1,0,0,0,13,8,1,0,1,0,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_hard.insert(max_padding,max_padding,en_labels_train,True)
df_hard.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0,1049,10,0,1,0,0,0,26,5,1,0,1,0,0,22,4,0,1,0,0,0,20,2,0,1,0,0,0,30,6,1,0,1,0,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,1773,4,1,0,1,0,0,13,7,1,0,1,0,0,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1375,5,0,1,0,0,0,21,5,0,1,0,0,0,24,7,1,0,1,0,0,16,4,0,1,0,0,0,27,8,1,0,1,0,0,20,2,0,1,0,0,0,18,2,1,0,1,0,0,22,0,0,0,0,0,0,0,0,0,0
3,0,1077,9,1,0,1,0,0,23,10,0,1,0,0,0,22,3,1,0,1,0,0,21,4,1,0,1,0,0,23,3,0,1,0,0,0,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,797,10,0,1,0,0,0,10,7,0,1,0,0,0,10,9,1,0,1,0,0,20,3,0,1,0,0,0,23,10,0,1,0,0,0,13,8,1,0,1,0,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_hard = df_hard.iloc[:,:-1]
X_hard.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,0,1049,10,0,1,0,0,0,26,5,1,0,1,0,0,22,4,0,1,0,0,0,20,2,0,1,0,0,0,30,6,1,0,1,0,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1773,4,1,0,1,0,0,13,7,1,0,1,0,0,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1375,5,0,1,0,0,0,21,5,0,1,0,0,0,24,7,1,0,1,0,0,16,4,0,1,0,0,0,27,8,1,0,1,0,0,20,2,0,1,0,0,0,18,2,1,0,1,0,0,22,0,0,0,0,0,0,0,0,0
3,0,1077,9,1,0,1,0,0,23,10,0,1,0,0,0,22,3,1,0,1,0,0,21,4,1,0,1,0,0,23,3,0,1,0,0,0,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,797,10,0,1,0,0,0,10,7,0,1,0,0,0,10,9,1,0,1,0,0,20,3,0,1,0,0,0,23,10,0,1,0,0,0,13,8,1,0,1,0,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
Y_hard = df_hard.iloc[:,-1]
Y_hard.head()

0    1
1    0
2    0
3    1
4    0
Name: 60, dtype: int64

## X,Y Train Test Split 

In [ ]:
X_train_hard, X_test_hard, Y_train_hard, Y_test_hard = train_test_split(X_hard, Y_hard, stratify=Y_hard,random_state=1,test_size=0.2)

In [ ]:
print("Train set X size: ",X_train_hard.shape)
print("Train set Y size: ",Y_train_hard.shape)
print("Test set X size: ",X_test_hard.shape)
print("Test set Y size: ",Y_test_hard.shape)

Train set X size:  (12000, 60)
Train set Y size:  (12000,)
Test set X size:  (3000, 60)
Test set Y size:  (3000,)


# Random Forest Classifier

## Random Forest Classifier for Easy Dataset

In [ ]:
clf_easy_rf = RandomForestClassifier()

In [ ]:
clf_easy_rf.fit(X_train_easy, Y_train_easy)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
Y_pred_easy = clf_easy_rf.predict(X_test_easy)

### Random Forest Accuracy for Easy Dataset

In [ ]:
accuracy_score(Y_test_easy,Y_pred_easy)

0.511

In [ ]:
X_test_easy.iloc[0].values

array([  0, 450,   6,   1,   0,   1,   0,   0,  28,   2,   0,   1,   0,
         0,   0,  18,   9,   0,   1,   0,   0,   0,  11,   8,   1,   0,
         1,   0,   0,  10,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [ ]:
clf_easy_rf.classes_

array([0, 1])

In [ ]:
Y_pred_easy.shape

(3000,)

### Save Model

In [ ]:
with open("./easy-rf.pkl",'wb') as f:
    pickle.dump(clf_easy_rf,f)

In [ ]:
# with open("./easy-rf.pkl",'rb') as f:
#     pkl_rf = pickle.load(f)

## Random Forest Classifier for Medium Dataset

In [ ]:
clf_medium_rf = RandomForestClassifier()

In [ ]:
clf_medium_rf.fit(X_train_medium, Y_train_medium)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
Y_pred_medium = clf_medium_rf.predict(X_test_medium)

### Random Forest Accuracy for Medium Dataset

In [ ]:
accuracy_score(Y_test_medium,Y_pred_medium)

0.507

In [ ]:
with open("./medium-rf.pkl",'wb') as f:
    pickle.dump(clf_medium_rf,f)

## Random Forest Classifier for Hard Dataset

In [ ]:
clf_hard_rf = RandomForestClassifier()

In [ ]:
clf_hard_rf.fit(X_train_hard, Y_train_hard)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
Y_pred_hard = clf_hard_rf.predict(X_test_hard)

### Random Forest Accuracy for Hard Dataset

In [ ]:
accuracy_score(Y_test_hard,Y_pred_hard)

0.506

In [ ]:
with open("./hard-rf.pkl",'wb') as f:
    pickle.dump(clf_hard_rf,f)

## Prepping Train Datasets for 4th Model

In [ ]:
Y_preds_easy_4 = clf_easy_rf.predict_proba(df_easy_4)

In [ ]:
Y_preds_medium_4 = clf_medium_rf.predict_proba(df_medium_4)

In [ ]:
Y_preds_hard_4 = clf_hard_rf.predict_proba(df_hard_4)

In [ ]:
Y_preds_easy_4[:,0]

array([0.46, 0.53, 0.51, ..., 0.5 , 0.45, 0.43])

In [ ]:
Y_preds_medium_4

array([[0.45, 0.55],
       [0.49, 0.51],
       [0.5 , 0.5 ],
       ...,
       [0.5 , 0.5 ],
       [0.58, 0.42],
       [0.36, 0.64]])

In [ ]:
Y_preds_hard_4

array([[0.46, 0.54],
       [0.48, 0.52],
       [0.52, 0.48],
       ...,
       [0.5 , 0.5 ],
       [0.48, 0.52],
       [0.46, 0.54]])

In [ ]:
X_4_rf = np.array([Y_preds_easy_4[:,0],Y_preds_easy_4[:,-1],Y_preds_medium_4[:,0],Y_preds_medium_4[:,-1],Y_preds_hard_4[:,0],Y_preds_hard_4[:,-1]])
X_4_rf.shape

(6, 15000)

In [ ]:
Y_4_rf = en_labels_4

In [ ]:
X_4_rf.shape,Y_4_rf.shape

((6, 15000), (15000,))

In [ ]:
X_4_rf = X_4_rf.transpose()

In [ ]:
X_4_rf.shape,Y_4_rf.shape

((15000, 6), (15000,))

In [ ]:
X_train_4, X_test_4, Y_train_4, Y_test_4 = train_test_split(X_4_rf, Y_4_rf, stratify=Y_4_rf,random_state=1,test_size=0.2)

In [ ]:
print("Train set X size: ",X_train_4.shape)
print("Train set Y size: ",Y_train_4.shape)
print("Test set X size: ",X_test_4.shape)
print("Test set Y size: ",Y_test_4.shape)

Train set X size:  (12000, 6)
Train set Y size:  (12000,)
Test set X size:  (3000, 6)
Test set Y size:  (3000,)


## Random Forest Classifier for 4th Model

In [ ]:
clf_4_rf = RandomForestClassifier()

In [ ]:
clf_4_rf.fit(X_train_4,Y_train_4)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
Y_pred_4 = clf_4_rf.predict(X_test_4)

### Final Accuracy for Random Forest Classifier

In [ ]:
accuracy_score(Y_test_4,Y_pred_4)

0.509

In [ ]:
with open("./final-rf.pkl",'wb') as f:
    pickle.dump(clf_4_rf,f)

In [ ]:
clf_4_rf.predict(X_test_4)

array([1, 0, 1, ..., 1, 0, 1])

In [ ]:
clf_4_rf.classes_

array([0, 1])

In [ ]:
for p in clf_4_rf.predict_proba(X_test_4):
    print(p)

[0.45833333 0.54166667]
[0.55 0.45]
[0.25166667 0.74833333]
[0.31 0.69]
[0.1775 0.8225]
[0.95 0.05]
[0.584 0.416]
[0.455 0.545]
[0.58666667 0.41333333]
[0.47225 0.52775]
[0.55 0.45]
[0.86 0.14]
[0.44133333 0.55866667]
[0.32083333 0.67916667]
[0.185 0.815]
[0.50811905 0.49188095]
[0.06 0.94]
[0.23 0.77]
[0.03 0.97]
[0.46 0.54]
[0.23833333 0.76166667]
[0.565 0.435]
[0.48 0.52]
[0.4035 0.5965]
[0.19358333 0.80641667]
[0.20466667 0.79533333]
[0.7975 0.2025]
[0.31 0.69]
[0.76166667 0.23833333]
[0.175 0.825]
[0.02166667 0.97833333]
[0.53016667 0.46983333]
[0.86 0.14]
[0.49 0.51]
[0.60666667 0.39333333]
[0.335 0.665]
[0.62698485 0.37301515]
[0.429 0.571]
[0.56 0.44]
[0.7225 0.2775]
[0.603 0.397]
[0.86583333 0.13416667]
[0.0675 0.9325]
[0.27 0.73]
[0.39 0.61]
[0.6 0.4]
[0.59641667 0.40358333]
[0.74083333 0.25916667]
[0.53766667 0.46233333]
[0.28916667 0.71083333]
[0.43 0.57]
[0.55333333 0.44666667]
[0.49 0.51]
[0.41883333 0.58116667]
[0.725 0.275]
[0.835 0.165]
[0.13 0.87]
[0.35 0.65]
[0.69 0.

# K-Nearest Neighbour Classifier

## KNN for Easy Dataset

In [ ]:
knn_easy = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn_easy.fit(X_train_easy, Y_train_easy)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

In [ ]:
Y_pred_easy = knn_easy.predict(X_test_easy)

### KNN Accuracy for Easy Dataset

In [ ]:
accuracy_score(Y_test_easy,Y_pred_easy)

0.5006666666666667

In [ ]:
with open("./easy-knn.pkl",'wb') as f:
    pickle.dump(knn_easy,f)

## KNN for Medium Dataset

In [ ]:
knn_medium = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn_medium.fit(X_train_medium, Y_train_medium)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

In [ ]:
Y_pred_medium = knn_medium.predict(X_test_medium)

### KNN Accuracy for Medium Dataset

In [ ]:
accuracy_score(Y_test_medium,Y_pred_medium)

0.5113333333333333

In [ ]:
with open("./medium-knn.pkl",'wb') as f:
    pickle.dump(knn_medium,f)

## KNN for Hard Dataset

In [ ]:
knn_hard= KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn_hard.fit(X_train_hard, Y_train_hard)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

In [ ]:
Y_pred_hard = knn_hard.predict(X_test_hard)

In [ ]:
with open("./hard-knn.pkl",'wb') as f:
    pickle.dump(knn_hard,f)

### KNN Accuracy for Hard Dataset

## Prepping Train Datasets for 4th KNN Model

In [ ]:
Y_preds_easy_4 = knn_easy.predict_proba(df_easy_4)

In [ ]:
Y_preds_medium_4 = knn_medium.predict_proba(df_medium_4)

In [ ]:
Y_preds_hard_4 = knn_hard.predict_proba(df_hard_4)

In [ ]:
Y_preds_easy_4

array([[0.42857143, 0.57142857],
       [0.57142857, 0.42857143],
       [0.42857143, 0.57142857],
       ...,
       [0.42857143, 0.57142857],
       [0.71428571, 0.28571429],
       [0.42857143, 0.57142857]])

In [ ]:
Y_preds_medium_4

array([[0.57142857, 0.42857143],
       [0.28571429, 0.71428571],
       [0.42857143, 0.57142857],
       ...,
       [0.42857143, 0.57142857],
       [0.57142857, 0.42857143],
       [0.85714286, 0.14285714]])

In [ ]:
Y_preds_hard_4

array([[0.57142857, 0.42857143],
       [0.42857143, 0.57142857],
       [0.71428571, 0.28571429],
       ...,
       [0.14285714, 0.85714286],
       [0.42857143, 0.57142857],
       [0.57142857, 0.42857143]])

In [ ]:
X_4_knn = np.array([Y_preds_easy_4[:,0],Y_preds_easy_4[:,-1],Y_preds_medium_4[:,0],Y_preds_medium_4[:,-1],Y_preds_hard_4[:,0],Y_preds_hard_4[:,-1]])
X_4_knn.shape

(6, 15000)

In [ ]:
X_4_knn

array([[0.42857143, 0.57142857, 0.42857143, ..., 0.42857143, 0.71428571,
        0.42857143],
       [0.57142857, 0.42857143, 0.57142857, ..., 0.57142857, 0.28571429,
        0.57142857],
       [0.57142857, 0.28571429, 0.42857143, ..., 0.42857143, 0.57142857,
        0.85714286],
       [0.42857143, 0.71428571, 0.57142857, ..., 0.57142857, 0.42857143,
        0.14285714],
       [0.57142857, 0.42857143, 0.71428571, ..., 0.14285714, 0.42857143,
        0.57142857],
       [0.42857143, 0.57142857, 0.28571429, ..., 0.85714286, 0.57142857,
        0.42857143]])

In [ ]:
Y_4_knn = en_labels_4

In [ ]:
X_4_knn.shape,Y_4_knn.shape

((6, 15000), (15000,))

In [ ]:
X_4_knn = X_4_knn.transpose()

In [ ]:
X_4_knn

array([[0.42857143, 0.57142857, 0.57142857, 0.42857143, 0.57142857,
        0.42857143],
       [0.57142857, 0.42857143, 0.28571429, 0.71428571, 0.42857143,
        0.57142857],
       [0.42857143, 0.57142857, 0.42857143, 0.57142857, 0.71428571,
        0.28571429],
       ...,
       [0.42857143, 0.57142857, 0.42857143, 0.57142857, 0.14285714,
        0.85714286],
       [0.71428571, 0.28571429, 0.57142857, 0.42857143, 0.42857143,
        0.57142857],
       [0.42857143, 0.57142857, 0.85714286, 0.14285714, 0.57142857,
        0.42857143]])

In [ ]:
X_4_knn.shape,Y_4_knn.shape

((15000, 6), (15000,))

In [ ]:
X_train_4, X_test_4, Y_train_4, Y_test_4 = train_test_split(X_4_knn, Y_4_knn, stratify=Y_4_knn,random_state=1,test_size=0.2)

In [ ]:
print("Train set X size: ",X_train_4.shape)
print("Train set Y size: ",Y_train_4.shape)
print("Test set X size: ",X_test_4.shape)
print("Test set Y size: ",Y_test_4.shape)

Train set X size:  (12000, 6)
Train set Y size:  (12000,)
Test set X size:  (3000, 6)
Test set Y size:  (3000,)


## KNN for 4th Model

In [ ]:
knn_4 = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn_4.fit(X_train_4,Y_train_4)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

In [ ]:
Y_pred_4 = knn_4.predict(X_test_4)

### Final Accuracy for KNN Classifier

In [ ]:
accuracy_score(Y_test_4,Y_pred_4)

0.5006666666666667

In [ ]:
with open("./final-knn.pkl",'wb') as f:
    pickle.dump(knn_4,f)

# SVM Classifier

## SVM for Easy Dataset

In [ ]:
svm_easy = SVC(probability=True)

In [ ]:
svm_easy.fit(X_train_easy, Y_train_easy)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
Y_pred_easy = svm_easy.predict(X_test_easy)

### SVM Accuracy for Easy Dataset

In [ ]:
accuracy_score(Y_test_easy,Y_pred_easy)

0.491

In [ ]:
with open("./easy-svm.pkl",'wb') as f:
    pickle.dump(svm_easy,f)

## SVM for Medium Dataset

In [ ]:
svm_medium = SVC(probability=True)

In [ ]:
svm_medium.fit(X_train_medium, Y_train_medium)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
Y_pred_medium = svm_medium.predict(X_test_medium)

### SVM Accuracy for Medium Dataset

In [ ]:
accuracy_score(Y_test_medium,Y_pred_medium)

0.491

In [ ]:
with open("./medium-svm.pkl",'wb') as f:
    pickle.dump(svm_medium,f)

In [ ]:
X_test_medium.shape

(3000, 60)

In [ ]:
X_test_medium[0].shape

(3000,)

## SVM for Hard Dataset

In [ ]:
svm_hard = SVC(probability=True)

In [ ]:
svm_hard.fit(X_train_hard, Y_train_hard)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
Y_pred_hard = svm_hard.predict(X_test_hard)

### SVM Accuracy for Hard Dataset

In [ ]:
accuracy_score(Y_test_hard,Y_pred_hard)

0.49

In [ ]:
with open("./hard-svm.pkl",'wb') as f:
    pickle.dump(svm_hard,f)

## Prepping Train Datasets for 4th SVM Model

In [ ]:
Y_preds_easy_4 = svm_easy.predict_proba(df_easy_4)

In [ ]:
Y_preds_medium_4 = svm_medium.predict_proba(df_medium_4)

In [ ]:
Y_preds_hard_4 = svm_hard.predict_proba(df_hard_4)

In [ ]:
Y_preds_easy_4

array([[0.5       , 0.5       ],
       [0.5       , 0.5       ],
       [0.5       , 0.5       ],
       ...,
       [0.48963653, 0.51036347],
       [0.49129844, 0.50870156],
       [0.50516931, 0.49483069]])

In [ ]:
Y_preds_medium_4

array([[0.5      , 0.5      ],
       [0.5      , 0.5      ],
       [0.5      , 0.5      ],
       ...,
       [0.4910691, 0.5089309],
       [0.4926089, 0.5073911],
       [0.5      , 0.5      ]])

In [ ]:
Y_preds_hard_4

array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       ...,
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]])

In [ ]:
X_4_svm = np.array([Y_preds_easy_4[:,0],Y_preds_easy_4[:,-1],Y_preds_medium_4[:,0],Y_preds_medium_4[:,-1],Y_preds_hard_4[:,0],Y_preds_hard_4[:,-1]])
X_4_svm.shape

(6, 15000)

In [ ]:
X_4_svm

array([[0.5       , 0.5       , 0.5       , ..., 0.48963653, 0.49129844,
        0.50516931],
       [0.5       , 0.5       , 0.5       , ..., 0.51036347, 0.50870156,
        0.49483069],
       [0.5       , 0.5       , 0.5       , ..., 0.4910691 , 0.4926089 ,
        0.5       ],
       [0.5       , 0.5       , 0.5       , ..., 0.5089309 , 0.5073911 ,
        0.5       ],
       [0.5       , 0.5       , 0.5       , ..., 0.5       , 0.5       ,
        0.5       ],
       [0.5       , 0.5       , 0.5       , ..., 0.5       , 0.5       ,
        0.5       ]])

In [ ]:
Y_4_svm = en_labels_4

In [ ]:
X_4_svm.shape,Y_4_svm.shape

((6, 15000), (15000,))

In [ ]:
X_4_svm = X_4_svm.transpose()

In [ ]:
X_4_svm.shape,Y_4_svm.shape

((15000, 6), (15000,))

In [ ]:
X_4_svm

array([[0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       ],
       [0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       ],
       [0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
        0.5       ],
       ...,
       [0.48963653, 0.51036347, 0.4910691 , 0.5089309 , 0.5       ,
        0.5       ],
       [0.49129844, 0.50870156, 0.4926089 , 0.5073911 , 0.5       ,
        0.5       ],
       [0.50516931, 0.49483069, 0.5       , 0.5       , 0.5       ,
        0.5       ]])

In [ ]:
X_train_4, X_test_4, Y_train_4, Y_test_4 = train_test_split(X_4_svm, Y_4_svm, stratify=Y_4_svm,random_state=1,test_size=0.2)

In [ ]:
print("Train set X size: ",X_train_4.shape)
print("Train set Y size: ",Y_train_4.shape)
print("Test set X size: ",X_test_4.shape)
print("Test set Y size: ",Y_test_4.shape)

Train set X size:  (12000, 6)
Train set Y size:  (12000,)
Test set X size:  (3000, 6)
Test set Y size:  (3000,)


## SVM for 4th Model

In [ ]:
svm_4 = SVC(probability=True)

In [ ]:
svm_4.fit(X_train_4,Y_train_4)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
Y_pred_4 = svm_4.predict(X_test_4)

### Final Accuracy for SVM Classifier

In [ ]:
accuracy_score(Y_test_4,Y_pred_4)

0.5003333333333333

In [ ]:
with open("./final-svm.pkl",'wb') as f:
    pickle.dump(svm_4,f)

# Keras-Simple Neural Network

## Simple NN for Easy Dataset

In [ ]:
model_easy = Sequential()
model_easy.add(Dense(units=30,input_dim=60,activation='relu',kernel_initializer='glorot_uniform'))
# model_easy.add(Dense(units=30,activation='relu',kernel_initializer='glorot_uniform'))
model_easy.add(Dense(units=15,activation='relu',kernel_initializer='glorot_uniform'))
model_easy.add(Dense(1,activation='sigmoid',kernel_initializer='glorot_uniform'))
model_easy.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history_easy = model_easy.fit(X_train_easy,Y_train_easy,batch_size=100,epochs=30,verbose=True)

Epoch 1/30
120/120 [==============================] - 1s 2ms/step - loss: 4.5330 - accuracy: 0.4856
Epoch 2/30
120/120 [==============================] - 0s 2ms/step - loss: 0.8033 - accuracy: 0.4929
Epoch 3/30
120/120 [==============================] - 0s 2ms/step - loss: 0.7648 - accuracy: 0.5043
Epoch 4/30
120/120 [==============================] - 0s 2ms/step - loss: 0.7524 - accuracy: 0.5073
Epoch 5/30
120/120 [==============================] - 0s 2ms/step - loss: 0.7691 - accuracy: 0.5023
Epoch 6/30
120/120 [==============================] - 0s 2ms/step - loss: 0.7465 - accuracy: 0.5042
Epoch 7/30
120/120 [==============================] - 0s 2ms/step - loss: 0.7255 - accuracy: 0.4999
Epoch 8/30
120/120 [==============================] - 0s 2ms/step - loss: 0.7623 - accuracy: 0.4955
Epoch 9/30
120/120 [==============================] - 0s 2ms/step - loss: 0.7234 - accuracy: 0.5014
Epoch 10/30
120/120 [==============================] - 0s 2ms/step - loss: 0.7173 - accuracy: 0.5031

In [ ]:
loss,accuracy = model_easy.evaluate(X_test_easy,Y_test_easy)
print("Loss: {}, Accuracy: {}".format(loss,accuracy))

94/94 [==============================] - 0s 2ms/step - loss: 0.6981 - accuracy: 0.4993
Loss: 0.6980833411216736, Accuracy: 0.49933332204818726


In [ ]:
model_json = model_easy.to_json()
with open("nn-easy.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_easy.save_weights("nn-easy-weights.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
X_test_easy.iloc[0].values.reshape(1,-1).shape

(1, 60)

In [ ]:
predictions = model_easy.predict(X_test_easy.iloc[1].values.reshape(1,-1))
predictions

array([[0.40434366]], dtype=float32)

In [ ]:
1-predictions

array([[0.59565634]], dtype=float32)

In [ ]:
predictions = model_easy.predict(X_test_easy)
predictions

array([[0.51564425],
       [0.40434113],
       [0.45246714],
       ...,
       [0.41290507],
       [0.45665988],
       [0.48429888]], dtype=float32)

## Simple NN for Medium Dataset

In [ ]:
model_medium = Sequential()
model_medium.add(Dense(units=30,input_dim=60,activation='relu',kernel_initializer='glorot_uniform'))
# model_medium.add(Dense(units=30,activation='relu',kernel_initializer='glorot_uniform'))
model_medium.add(Dense(units=15,activation='relu',kernel_initializer='glorot_uniform'))
model_medium.add(Dense(1,activation='sigmoid',kernel_initializer='glorot_uniform'))
model_medium.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history_medium = model_medium.fit(X_train_medium,Y_train_medium,batch_size=100,epochs=30,verbose=True)

Epoch 1/30
120/120 [==============================] - 1s 2ms/step - loss: 2.2734 - accuracy: 0.5017
Epoch 2/30
120/120 [==============================] - 0s 2ms/step - loss: 0.7578 - accuracy: 0.4983
Epoch 3/30
120/120 [==============================] - 0s 2ms/step - loss: 0.7343 - accuracy: 0.4962
Epoch 4/30
120/120 [==============================] - 0s 2ms/step - loss: 0.7471 - accuracy: 0.4962
Epoch 5/30
120/120 [==============================] - 0s 2ms/step - loss: 0.7302 - accuracy: 0.5104
Epoch 6/30
120/120 [==============================] - 0s 2ms/step - loss: 0.7249 - accuracy: 0.5046
Epoch 7/30
120/120 [==============================] - 0s 2ms/step - loss: 0.7137 - accuracy: 0.4980
Epoch 8/30
120/120 [==============================] - 0s 2ms/step - loss: 0.7093 - accuracy: 0.5177
Epoch 9/30
120/120 [==============================] - 0s 2ms/step - loss: 0.7225 - accuracy: 0.5024
Epoch 10/30
120/120 [==============================] - 0s 2ms/step - loss: 0.7249 - accuracy: 0.5028

In [ ]:
loss,accuracy = model_medium.evaluate(X_test_medium,Y_test_medium)
print("Loss: {}, Accuracy: {}".format(loss,accuracy))

94/94 [==============================] - 0s 2ms/step - loss: 0.7249 - accuracy: 0.4993
Loss: 0.7248897552490234, Accuracy: 0.49933332204818726


In [ ]:
X_test_medium.iloc[0].values.reshape(1,-1).shape

(1, 60)

In [ ]:
predictions = model_medium.predict(X_test_medium.iloc[1].values.reshape(1,-1))
predictions

array([[0.31930616]], dtype=float32)

In [ ]:
model_json = model_medium.to_json()
with open("nn-medium.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_medium.save_weights("nn-medium-weights.h5")
print("Saved model to disk")

Saved model to disk


## Simple NN for Hard Dataset

In [ ]:
model_hard = Sequential()
model_hard.add(Dense(units=30,input_dim=60,activation='relu',kernel_initializer='glorot_uniform'))
# model_hard.add(Dense(units=30,activation='relu',kernel_initializer='glorot_uniform'))
model_hard.add(Dense(units=15,activation='relu',kernel_initializer='glorot_uniform'))
model_hard.add(Dense(1,activation='sigmoid',kernel_initializer='glorot_uniform'))
model_hard.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history_hard = model_hard.fit(X_train_hard,Y_train_hard,batch_size=32,epochs=100,verbose=True)

Epoch 1/100
375/375 [==============================] - 1s 2ms/step - loss: 8.2240 - accuracy: 0.5102
Epoch 2/100
375/375 [==============================] - 1s 2ms/step - loss: 0.8565 - accuracy: 0.5056
Epoch 3/100
375/375 [==============================] - 1s 2ms/step - loss: 0.8029 - accuracy: 0.5029
Epoch 4/100
375/375 [==============================] - 1s 2ms/step - loss: 0.8259 - accuracy: 0.4950
Epoch 5/100
375/375 [==============================] - 1s 2ms/step - loss: 0.8703 - accuracy: 0.4929
Epoch 6/100
375/375 [==============================] - 1s 2ms/step - loss: 0.8346 - accuracy: 0.4852
Epoch 7/100
375/375 [==============================] - 1s 2ms/step - loss: 0.8188 - accuracy: 0.4979
Epoch 8/100
375/375 [==============================] - 1s 2ms/step - loss: 0.8076 - accuracy: 0.5031
Epoch 9/100
375/375 [==============================] - 1s 2ms/step - loss: 0.8076 - accuracy: 0.5040
Epoch 10/100
375/375 [==============================] - 1s 2ms/step - loss: 0.8346 - accura

In [ ]:
loss,accuracy = model_hard.evaluate(X_test_hard,Y_test_hard)
print("Loss: {}, Accuracy: {}".format(loss,accuracy))

94/94 [==============================] - 0s 2ms/step - loss: 0.6968 - accuracy: 0.4980
Loss: 0.6968071460723877, Accuracy: 0.49799999594688416


In [ ]:
X_test_hard.iloc[0].values.reshape(1,-1).shape

(1, 60)

In [ ]:
predictions = model_hard.predict(X_test_hard)
predictions

array([[0.33176768],
       [0.42714664],
       [0.47364938],
       ...,
       [0.4632027 ],
       [0.4498628 ],
       [0.4527891 ]], dtype=float32)

In [ ]:
model_json = model_hard.to_json()
with open("nn-hard.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_hard.save_weights("nn-hard-weights.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
predictions[:,0]

array([0.33176768, 0.42714664, 0.47364938, ..., 0.4632027 , 0.4498628 ,
       0.4527891 ], dtype=float32)

## Prepping Data for 4th NN Model

In [ ]:
Y_preds_easy_4 = model_easy.predict(df_easy_4)

In [ ]:
Y_preds_medium_4 = model_medium.predict(df_medium_4)

In [ ]:
Y_preds_hard_4 = model_hard.predict(df_hard_4)

In [ ]:
Y_preds_easy_4

array([[0.45534056],
       [0.5222685 ],
       [0.40881902],
       ...,
       [0.50456405],
       [0.5261788 ],
       [0.50648695]], dtype=float32)

In [ ]:
Y_preds_medium_4

array([[0.3408947 ],
       [0.33905432],
       [0.37069774],
       ...,
       [0.53500247],
       [0.44385645],
       [0.38541254]], dtype=float32)

In [ ]:
Y_preds_hard_4

array([[0.46975866],
       [0.47599098],
       [0.45403102],
       ...,
       [0.50492966],
       [0.4823984 ],
       [0.44151852]], dtype=float32)

In [ ]:
X_4_nn = np.array([Y_preds_easy_4[:,0],Y_preds_medium_4[:,0],Y_preds_hard_4[:,0]])
X_4_nn.shape

(3, 15000)

In [ ]:
X_4_nn

array([[0.45534056, 0.5222685 , 0.40881902, ..., 0.50456405, 0.5261788 ,
        0.50648695],
       [0.3408947 , 0.33905432, 0.37069774, ..., 0.53500247, 0.44385645,
        0.38541254],
       [0.46975866, 0.47599098, 0.45403102, ..., 0.50492966, 0.4823984 ,
        0.44151852]], dtype=float32)

In [ ]:
Y_4_nn = en_labels_4

In [ ]:
X_4_nn.shape,Y_4_nn.shape

((3, 15000), (15000,))

In [ ]:
X_4_nn = X_4_nn.transpose()

In [ ]:
X_4_nn.shape,Y_4_nn.shape

((15000, 3), (15000,))

In [ ]:
X_4_nn

array([[0.45534056, 0.3408947 , 0.46975866],
       [0.5222685 , 0.33905432, 0.47599098],
       [0.40881902, 0.37069774, 0.45403102],
       ...,
       [0.50456405, 0.53500247, 0.50492966],
       [0.5261788 , 0.44385645, 0.4823984 ],
       [0.50648695, 0.38541254, 0.44151852]], dtype=float32)

In [ ]:
X_train_4, X_test_4, Y_train_4, Y_test_4 = train_test_split(X_4_nn, Y_4_nn, stratify=Y_4_nn,random_state=1,test_size=0.2)

In [ ]:
print("Train set X size: ",X_train_4.shape)
print("Train set Y size: ",Y_train_4.shape)
print("Test set X size: ",X_test_4.shape)
print("Test set Y size: ",Y_test_4.shape)
X_train_4
# Y_train_4

Train set X size:  (12000, 3)
Train set Y size:  (12000,)
Test set X size:  (3000, 3)
Test set Y size:  (3000,)


array([[0.47851342, 0.38895246, 0.44964534],
       [0.44568002, 0.3309017 , 0.48213273],
       [0.4992176 , 0.41771394, 0.47827092],
       ...,
       [0.4464308 , 0.47214904, 0.4867093 ],
       [0.46977642, 0.4398732 , 0.48262793],
       [0.45764995, 0.38406163, 0.47164005]], dtype=float32)

## Simple NN for Final Model

In [ ]:
model_4 = Sequential()
# model_4.add(Dense(units=3,,activation='relu',kernel_initializer='glorot_uniform'))
model_4.add(Dense(1,input_dim=3,activation='sigmoid',kernel_initializer='glorot_uniform'))
model_4.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history_4 = model_4.fit(X_train_4,Y_train_4,batch_size=120,epochs=200,verbose=True)

Epoch 1/200
100/100 [==============================] - 1s 2ms/step - loss: 0.8369 - accuracy: 0.4973
Epoch 2/200
100/100 [==============================] - 0s 2ms/step - loss: 0.7865 - accuracy: 0.4980
Epoch 3/200
100/100 [==============================] - 0s 2ms/step - loss: 0.7527 - accuracy: 0.4950
Epoch 4/200
100/100 [==============================] - 0s 2ms/step - loss: 0.7298 - accuracy: 0.4907
Epoch 5/200
100/100 [==============================] - 0s 2ms/step - loss: 0.7129 - accuracy: 0.4937
Epoch 6/200
100/100 [==============================] - 0s 2ms/step - loss: 0.7047 - accuracy: 0.4892
Epoch 7/200
100/100 [==============================] - 0s 2ms/step - loss: 0.6972 - accuracy: 0.5002
Epoch 8/200
100/100 [==============================] - 0s 2ms/step - loss: 0.6952 - accuracy: 0.4991
Epoch 9/200
100/100 [==============================] - 0s 2ms/step - loss: 0.6940 - accuracy: 0.4991
Epoch 10/200
100/100 [==============================] - 0s 2ms/step - loss: 0.6942 - accura

In [ ]:
loss,accuracy = model_4.evaluate(X_test_4,Y_test_4)
print("Loss: {}, Accuracy: {}".format(loss,accuracy))

94/94 [==============================] - 0s 2ms/step - loss: 0.6932 - accuracy: 0.4997
Loss: 0.6931630969047546, Accuracy: 0.49966666102409363


In [ ]:
X_test_4.shape

(3000, 3)

In [ ]:
predictions = model_4.predict(X_test_4)
predictions

array([[0.49627408],
       [0.5027094 ],
       [0.5001971 ],
       ...,
       [0.48951322],
       [0.49380296],
       [0.49973723]], dtype=float32)

In [ ]:
for p in  predictions:
    print(p)

[0.49627408]
[0.5027094]
[0.5001971]
[0.50031155]
[0.49984306]
[0.49244496]
[0.4960319]
[0.50009596]
[0.4972028]
[0.501756]
[0.4855231]
[0.5107556]
[0.50466037]
[0.4916106]
[0.49410173]
[0.49047983]
[0.5102188]
[0.5059773]
[0.50187176]
[0.49443746]
[0.49524912]
[0.5001299]
[0.5019337]
[0.49822652]
[0.5079128]
[0.50470185]
[0.50411993]
[0.4948891]
[0.49623936]
[0.49916852]
[0.4997311]
[0.49655032]
[0.4895639]
[0.48761225]
[0.49457225]
[0.4936508]
[0.4962544]
[0.4992551]
[0.49181563]
[0.49687433]
[0.5077566]
[0.4978822]
[0.4954364]
[0.50168335]
[0.4988677]
[0.5112948]
[0.49622828]
[0.49883294]
[0.49450997]
[0.50623894]
[0.49502707]
[0.49737903]
[0.5030214]
[0.50799555]
[0.49657196]
[0.5144908]
[0.5067491]
[0.50517297]
[0.49788177]
[0.5004355]
[0.5015475]
[0.49422562]
[0.49824515]
[0.49908105]
[0.49257317]
[0.49995828]
[0.4985559]
[0.50339025]
[0.49609128]
[0.49518955]
[0.4937897]
[0.49317345]
[0.49703425]
[0.5060284]
[0.49826944]
[0.4886515]
[0.5021594]
[0.50654846]
[0.49479723]
[0.49283

In [ ]:
model_json = model_4.to_json()
with open("nn-final.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_4.save_weights("nn-final-weights.h5")
print("Saved model to disk")

Saved model to disk


# Final Model Selection

The final model selection due to it's performance and easibility in usage is `Random Forest Classifier`

In [ ]:
clf_4_rf

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)